In [1]:
import pandas as pd
import numpy as np
import statistics
from itertools import islice
import time

devices_data = pd.read_csv('C:\\Users\\ruben\\Desktop\\Investigation\\updated_dataset_07_3_2018\\devices.csv', index_col='id', parse_dates=['created_at', 'updated_at'])
samples_data = pd.read_csv('C:\\Users\\ruben\\Desktop\\Investigation\\updated_dataset_07_3_2018\\samples.csv', usecols=['device_id', 'timestamp', 'battery_state', 'battery_level'], parse_dates=['timestamp'])

#clean fulls where battery level < 100% and charges where battery level = 100%
samples_data.drop(samples_data.loc[(samples_data['battery_state'] == 'Full') & (samples_data['battery_level'] < 1)].index, inplace=True)
samples_data.drop(samples_data.loc[(samples_data['battery_state'] == 'Charging') & (samples_data['battery_level'] == 1)].index, inplace=True)

print('Loaded!!')

Loaded!!


In [6]:
def computeChargeSampleStats(charge_subset):
    if (len(charge_subset) < 5) and (charge_subset.loc[0]['battery_level'] == charge_subset.loc[len(charge_subset) - 1]['battery_level']):
        return -1
    
    first_valid_index = 0
    last_valid_index = len(charge_subset) - 1
    found_last = False
    
    print(charge_subset)
    for index, row in charge_subset.iterrows():
        if charge_subset.loc[first_valid_index]['battery_level'] == charge_subset.loc[index]['battery_level']:
            first_valid_index = index
        
        if charge_subset.loc[last_valid_index]['battery_level'] == charge_subset.loc[index]['battery_level'] and not found_last:
            last_valid_index = index
            found_last = True
            
        if index != 0 and (charge_subset.loc[index]['battery_level'] - charge_subset.loc[index - 1]['battery_level']) * 100 > 4:
            first_valid_index = index
    
    
    if first_valid_index != last_valid_index:
        time_per_charge = (charge_subset.loc[last_valid_index]['timestamp'] - charge_subset.loc[first_valid_index]['timestamp']).seconds / ((charge_subset.loc[last_valid_index]['battery_level'] - charge_subset.loc[first_valid_index]['battery_level']) * 100)
       
        if time_per_charge < 600:
            print(time_per_charge)
            print(time.strftime("%M:%S", time.gmtime(time_per_charge)))
            print(first_valid_index)
            print(last_valid_index)

            return time_per_charge
        else: 
            return -1
    else:
        return -1
    
def computeDeviceKPIS(devID):
    #check if the device has samples
    if samples_data.loc[samples_data['device_id'] == devID].empty:
        print('NO SAMPLES!!')
        return -1
    else:
        device_charge_samples = samples_data.loc[(samples_data['device_id'] == devID) & (samples_data['battery_state'].isin(['Charging', 'Full']))].drop_duplicates(subset=['timestamp']).sort_values(by='timestamp').reset_index(drop=True)
#         device_discharge_samples = samples_data.loc[(samples_data['device_id'] == devID) & (samples_data['battery_state'] == 'Discharging')].drop_duplicates(subset=['timestamp']).sort_values(by='timestamp').reset_index(drop=True)
#         device_not_charging_samples = samples_data.loc[(samples_data['device_id'] == devID) & (samples_data['battery_state'] == 'Not charging')].drop_duplicates(subset=['timestamp']).sort_values(by='timestamp').reset_index(drop=True)
        
        charge_samples_results = []
        dicharge_samples_results = []
        
        if not device_charge_samples.empty:
            previous_index = 0  
            for index, row in islice(device_charge_samples.iterrows(), 1, None):
                #creates subsets
                if row['battery_level'] < device_charge_samples.iloc[index - 1].battery_level:
                    subset = device_charge_samples[previous_index:index].reset_index(drop=True)
#                     print(subset)

                    res = computeChargeSampleStats(subset)

                    if res != -1:
                        charge_samples_results.append(res)
                    
                    previous_index = index
                
                #last samples into subset
                if index == len(device_charge_samples) - 1:
                    subset = device_charge_samples[previous_index:index + 1].reset_index(drop=True)
#                     print(subset)

                    res = computeChargeSampleStats(subset)

                    if res != -1:
                        charge_samples_results.append(res)

                
#         print(charge_samples_results)
        
        if charge_samples_results:
            t = sum(charge_samples_results)/float(len(charge_samples_results))
            print(time.strftime("%M:%S", time.gmtime(t)))
        else:
            print('not enought')


computeDeviceKPIS(6)
# 33, 54, 68
# 250, 253, 257, 258
# for devID in devices_data.index:
#     print(devID)
#     computeDeviceKPIS(devID)
#     print('------')

   device_id           timestamp battery_state  battery_level
0          6 2017-10-09 16:29:05      Charging           0.14
1          6 2017-10-09 16:38:43      Charging           0.19
2          6 2017-10-09 16:40:07      Charging           0.20
3          6 2017-10-09 16:42:09      Charging           0.21
4          6 2017-10-09 16:43:37      Charging           0.22
5          6 2017-10-09 17:03:16      Charging           0.27
6          6 2017-10-09 17:04:10      Charging           0.28
7          6 2017-10-09 17:06:31      Charging           0.29
8          6 2017-10-09 17:07:36      Charging           0.30
9          6 2017-10-09 17:10:04      Charging           0.31
102.0
01:42
5
9
    device_id           timestamp battery_state  battery_level
0           6 2017-10-09 18:37:49      Charging           0.14
1           6 2017-10-09 18:38:25      Charging           0.15
2           6 2017-10-09 18:39:20      Charging           0.16
3           6 2017-10-09 18:40:03      Charging   

59.8352941176
00:59
0
85
    device_id           timestamp battery_state  battery_level
0           6 2017-10-10 15:08:10      Charging           0.17
1           6 2017-10-10 15:08:46      Charging           0.18
2           6 2017-10-10 15:10:15      Charging           0.19
3           6 2017-10-10 15:11:08      Charging           0.20
4           6 2017-10-10 15:12:19      Charging           0.21
5           6 2017-10-10 15:12:54      Charging           0.22
6           6 2017-10-10 15:13:51      Charging           0.23
7           6 2017-10-10 15:14:37      Charging           0.24
8           6 2017-10-10 15:15:48      Charging           0.25
9           6 2017-10-10 15:16:31      Charging           0.26
10          6 2017-10-10 15:17:37      Charging           0.27
11          6 2017-10-10 15:18:10      Charging           0.28
12          6 2017-10-10 15:19:33      Charging           0.29
13          6 2017-10-10 15:20:03      Charging           0.30
14          6 2017-10-10 15:21

0
1
   device_id           timestamp battery_state  battery_level
0          6 2017-10-16 17:54:02      Charging           0.16
1          6 2017-10-16 17:55:01      Charging           0.17
2          6 2017-10-16 17:56:00      Charging           0.18
3          6 2017-10-16 17:56:43      Charging           0.19
4          6 2017-10-16 17:57:32      Charging           0.20
5          6 2017-10-16 17:58:21      Charging           0.21
51.8
00:51
0
5
    device_id           timestamp battery_state  battery_level
0           6 2017-10-16 23:20:21      Charging           0.03
1           6 2017-10-16 23:21:03      Charging           0.04
2           6 2017-10-16 23:21:51      Charging           0.05
3           6 2017-10-16 23:22:33      Charging           0.06
4           6 2017-10-16 23:23:15      Charging           0.07
5           6 2017-10-16 23:23:57      Charging           0.08
6           6 2017-10-16 23:24:43      Charging           0.09
7           6 2017-10-16 23:25:22      Char

    device_id           timestamp battery_state  battery_level
0           6 2017-10-23 16:52:31      Charging           0.03
1           6 2017-10-23 16:53:26      Charging           0.04
2           6 2017-10-23 16:54:16      Charging           0.05
3           6 2017-10-23 16:54:56      Charging           0.06
4           6 2017-10-23 16:55:36      Charging           0.07
5           6 2017-10-23 16:56:19      Charging           0.08
6           6 2017-10-23 16:57:00      Charging           0.09
7           6 2017-10-23 16:57:46      Charging           0.10
8           6 2017-10-23 16:58:23      Charging           0.11
9           6 2017-10-23 16:59:17      Charging           0.12
10          6 2017-10-23 16:59:54      Charging           0.13
11          6 2017-10-23 17:00:36      Charging           0.14
12          6 2017-10-23 17:01:14      Charging           0.15
13          6 2017-10-23 17:01:59      Charging           0.16
14          6 2017-10-23 17:02:36      Charging        

3          6 2017-11-01 12:22:07      Charging           0.17
   device_id           timestamp battery_state  battery_level
0          6 2017-11-01 12:22:35      Charging           0.16
1          6 2017-11-01 12:22:42      Charging           0.17
7.0
00:06
0
1
   device_id           timestamp battery_state  battery_level
0          6 2017-11-01 12:25:04      Charging           0.16
1          6 2017-11-01 12:25:15      Charging           0.17
2          6 2017-11-01 12:31:03      Charging           0.18
3          6 2017-11-01 12:34:24      Charging           0.19
186.666666667
03:06
0
3
   device_id           timestamp battery_state  battery_level
0          6 2017-11-01 12:35:20      Charging           0.18
1          6 2017-11-01 12:36:04      Charging           0.19
44.0
00:43
0
1
   device_id           timestamp battery_state  battery_level
0          6 2017-11-01 12:36:56      Charging           0.18
1          6 2017-11-01 12:37:05      Charging           0.19
2          6 2017

140.0
02:19
4
5
    device_id           timestamp battery_state  battery_level
0           6 2017-11-15 15:36:15      Charging           0.07
1           6 2017-11-15 15:37:19      Charging           0.08
2           6 2017-11-16 15:14:19      Charging           0.08
3           6 2017-11-16 15:14:53      Charging           0.09
4           6 2017-11-16 15:15:50      Charging           0.10
5           6 2017-11-16 15:16:39      Charging           0.11
6           6 2017-11-16 15:17:53      Charging           0.12
7           6 2017-11-16 15:18:22      Charging           0.13
8           6 2017-11-16 15:19:07      Charging           0.14
9           6 2017-11-16 15:19:50      Charging           0.15
10          6 2017-11-16 15:20:40      Charging           0.16
11          6 2017-11-16 15:21:21      Charging           0.17
12          6 2017-11-16 15:22:08      Charging           0.18
13          6 2017-11-16 15:22:55      Charging           0.19
14          6 2017-11-16 15:23:42      

0
65
    device_id           timestamp battery_state  battery_level
0           6 2017-12-02 00:21:28      Charging           0.61
1           6 2017-12-02 00:23:02      Charging           0.62
2           6 2017-12-02 00:24:23      Charging           0.63
3           6 2017-12-02 00:25:54      Charging           0.64
4           6 2017-12-02 00:27:20      Charging           0.65
5           6 2017-12-02 00:28:44      Charging           0.66
6           6 2017-12-02 00:30:21      Charging           0.67
7           6 2017-12-02 00:31:45      Charging           0.68
8           6 2017-12-02 00:32:57      Charging           0.69
9           6 2017-12-02 00:33:55      Charging           0.70
10          6 2017-12-02 00:35:07      Charging           0.71
11          6 2017-12-02 00:36:13      Charging           0.72
12          6 2017-12-02 00:38:09      Charging           0.73
13          6 2017-12-02 00:39:13      Charging           0.74
14          6 2017-12-02 00:39:40      Charging   

C:\Users\ruben\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in double_scalars


In [16]:
a = samples_data.loc[(samples_data['device_id'] == 1 ) & (samples_data['battery_state'].isin(['Charging', 'Full']))].drop_duplicates(subset=['timestamp']).sort_values(by='timestamp').reset_index(drop=True)
a.head()

,device_id,timestamp,battery_state,battery_level
0,1,2017-10-08 13:14:22,Charging,0.80
1,1,2017-10-08 13:16:32,Charging,0.81
2,1,2017-10-08 13:18:37,Charging,0.82
3,1,2017-10-08 13:20:48,Charging,0.83
4,1,2017-10-08 13:24:24,Charging,0.85


In [7]:
import statistics

a = []


print(statistics.mean(a))
print(statistics.stdev(a))

StatisticsError: mean requires at least one data point

In [24]:
device_charge_samples = samples_data.loc[(samples_data['device_id'] == 6) & (samples_data['battery_state'].isin(['Charging', 'Full']))].drop_duplicates(subset=['timestamp']).sort_values(by='timestamp').reset_index(drop=True)

for r in islice(zip(device_charge_samples['timestamp'], device_charge_samples['battery_level']), 1, None):


0.19
0.2
0.21
0.22
0.27
0.28
0.29
0.3
0.31
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.15
0.16
0.17
0.18
0.19
0.2
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.16
0.17
0.18
0.19
0.2
0.

0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21

0.46
0.47
0.48
0.49
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
0.07
0.08
0.09
0.1
0.11
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.55
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.21
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84


In [22]:
device_charge_samples.head()

,device_id,timestamp,battery_state,battery_level
0,6,2017-10-09 16:29:05,Charging,0.14
1,6,2017-10-09 16:38:43,Charging,0.19
2,6,2017-10-09 16:40:07,Charging,0.20
3,6,2017-10-09 16:42:09,Charging,0.21
4,6,2017-10-09 16:43:37,Charging,0.22


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4021892 entries, 0 to 4036845
Data columns (total 4 columns):
device_id        int64
timestamp        datetime64[ns]
battery_state    object
battery_level    float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 153.4+ MB


array(9.0)